In [1]:
import json
from lxml import etree

In [4]:
# read data from the db
from database.connection import DatabaseConnection
engine = DatabaseConnection()
connection = engine.connect()

works = connection.execute(
    """
    SELECT *
    FROM ao3_scraper.works
    WHERE work_content_html IS NOT NULL
    ORDER BY kudos desc
    limit 20
    """
)

ModuleNotFoundError: No module named 'database'

In [2]:
tree = etree.parse('all_young_dudes.html', etree.HTMLParser())
root = tree.getroot()

### Book Title

In [3]:
book_data = {}

In [4]:
# title
book_title = root.xpath('head/title/text()')[0]
book_data['title'] = book_title
book_title

'All the Young Dudes - MsKingBean89 - Harry Potter - J. K. Rowling'

### Tags

I'm leaving these for now. It's formatted much better on the AO3 page. Quite crap to work with in the html download

In [5]:
# tags
tags = root.xpath("//div[@class='meta']//dl[@class='tags']")[0]

In [6]:
# TODO: handle tags and read hrefs
tag_labels = tags.xpath(".//dt//text()")
tag_values = tags.xpath(".//dd")

assert len(tag_labels) == len(tag_values)


In [7]:
# tag_data = []
# for label, values in zip(tag_labels, tag_values):

#     # clean up label
#     label = label[:-1] if label.endswith(':') else label
#     label = label.lower()
#     label = label.replace(' ', '_')

#     # will get stats from the main page
#     if label == 'stats':
#         continue

#     # convert values to list
#     # most already are but some (like language) aren't
#     if not isinstance(values, list):
#         values = [values]

#     values_list = []
#     for value in values:

#         attrs = {}

#         # most tag values are links so I can grab both the href and the link text
#         if value.tag == 'a':
#             attrs['text'] = value.text
#             attrs['link'] = value.get('href')

#         # some tag values are just text
#         if value.tag == 'dd':
#             attrs['text'] = value.text.strip()
#             attrs['link'] = None

#     tag_data.append({
#         'name': label,
#         'values': values_list
#     })

# tag_data

# test tags contain basic labels (language, stats)

# book_data['tags'] = tag_data

### Chapters

In [8]:
# create list of dicts for every chapter
book = root.xpath("//div[@id='chapters']")[0]

chapter_metadata = book.xpath(".//div[@class='meta group']") # //*[@class='heading']//text()")
chapter_content = book.xpath(".//div[@class='userstuff']")

# should have the same amount of chapter titles and content
assert len(chapter_metadata) == len(chapter_content)

chapters_data = []
for meta, content in zip(chapter_metadata, chapter_content):

    title = meta.xpath("./*[@class='heading']//text()")
    title = '\n'.join(title).strip()

    # everything in the meta group that isn't part of the chapter title
    summary_and_notes_raw = meta.xpath(".//text()[not(ancestor::*[@class='heading'])]")
    summary_and_notes_stripped = [item.strip() for item in summary_and_notes_raw]
    summary_and_notes = '\n'.join(summary_and_notes_stripped).strip()

    text_raw = content.xpath(".//text()")
    text_stripped = [item.strip() for item in text_raw]
    text = '\n'.join(text_stripped).strip()

    chapter_dict = {
        'title': title,
        'summary_and_notes': summary_and_notes,
        'content': text
    }

    chapters_data.append(chapter_dict)

book_data['chapters'] = chapters_data

print(f'{len(chapters_data)} chapters found')

188 chapters found


In [ ]:
# TODO: I'm missing chapter end notes. Might have to restructure the chapter data approach to get this
book_data

{'title': 'All the Young Dudes - MsKingBean89 - Harry Potter - J. K. Rowling',
 'chapters': [{'title': "Summer, 1971: St Edmund's",
   'summary_and_notes': '',
   'content': "Saturday 7th August, 1971\nHe woke up in the dark. It was too hot in the little room they’d put him in, being early August. Though he supposed that could be the fever. He always had a high temperature, the morning after. They used to put him in a room with a window, but a few months ago he’d been able to smash one of them, and if it hadn’t had bars anyway then he’d have escaped. He’d heard them talking about restraining him as he got older. He tried not to think about it.\nHe remembered the feeling of hunger, so intense it transformed into rage. He remembered howling and keening for hours, circling the cell over and over again. Perhaps they’d let him off lessons today, and he could sleep. It was the summer holiday’s anyway, and not fair that he had to do lessons when all the other boys were allowed to spend all da

In [10]:
# write data to json
with open('book_data.json', 'w') as f:
    json.dump(book_data, f)


### Tests

In [11]:
assert book_data['title'] != None

In [12]:
assert len(book_data['chapters']) > 0

In [13]:
for chapter in book_data['chapters']:

    assert chapter['title'] is not None
    assert chapter['title'].strip() != ''

    assert chapter['content'] is not None
    assert chapter['content'].strip() != ''


### Pre process text?
- Haaaaaapppp-yyyy birthday to you 
- Ensure no unecerray changes are made

## TTS

In [28]:
import os

import replicate
from dotenv import load_dotenv

load_dotenv('.env')

True

In [29]:
chapter = book_data['chapters'][0]

In [30]:
len(chapter['content'])

11142

In [ ]:
api = replicate.Client(api_token=os.environ["REPLICATE_API_TOKEN"])

output = api.run(
    "jaaari/kokoro-82m:f559560eb822dc509045f3921a1921234918b91739db4bf3daab2169b71c7a13",
    input={
        "text": chapter['content'],
        "speed": 1,
        "voice": "af_bella"
    }
)
print(output)

# to store
# audio length
# time taken
# model, voice, hyper parameters

https://replicate.delivery/czjl/ni3j0M4AiDpiMxqwFEf56T5330qfAsQQU0k6uTjbAiBdA3UUA/output.wav


In [ ]:
# Bark - really cool and realistic but takes ages to generate and VERY unstable. It skips sentences frequently.
# XTTS-v2 - Decent but voice goes shaky at times and a little robotic (no more than koroko)

In [ ]:
# TODO:
# create scraper to get all works
# get link to all works
# scrape works html